# Horario de mayor concurrencia de negocio


Script para hacer scraping del publico de un negocio en base a la altura de la barra del grafico que se muestra en la busqueda de google.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import warnings
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

import folium

import time, re

In [2]:
url = 'https://www.google.com/search?q=cordoba+shopping&oq=cordoba+shoop&aqs=edge.1.69i57j0i10i131i433i512j0i10i512l7.6107j0j1&sourceid=chrome&ie=UTF-8'

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

[WDM] - Downloading: 100%|██████████| 8.08M/8.08M [00:00<00:00, 20.1MB/s]


In [4]:
data = driver.find_elements_by_css_selector('div[role="radiogroup"]')

In [5]:
data = driver.find_elements_by_css_selector('div[data-hour]')

In [6]:
i    = 0
k    = 0
df   = pd.DataFrame()
df2  = pd.DataFrame()
df3  = pd.DataFrame()

In [7]:
def scrap(df, df2, data, i, k, n):
    for hora in data:
        try:
            df   = df.append( pd.Series(data[i].get_attribute('data-hour'))  , ignore_index=True)
            i    = i+1
        except:
            print('Hubo un error')
    for hora in data:
        try:
            df2  = df2.append( pd.Series(data[k].find_elements_by_css_selector('div')[1].get_attribute('style')) , ignore_index=True) 
            k = k + 1
        except:
            print('Hubo un error')
    df2 = (df2[0].str.split(expand=True)[1]).str.split('p', expand=True)[0].str.split(expand=True)
    df.columns=['Horas']
    df2.columns=['Cantidad']
    df3 = pd.concat([df, df2], axis=1)
    df3 = df3.assign( Dia  = n)
    return df3

In [8]:
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()= 'lun' ]"))).click()
df3 = df3.append(scrap(df, df2, data, i, k, 1))
i    = 0
k    = 0

In [9]:
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()= 'mar' ]"))).click()
df3 = df3.append(scrap(df, df2, data, i, k, 2))
i    = 0
k    = 0

In [10]:
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()= 'mié' ]"))).click()
df3 = df3.append(scrap(df, df2, data, i, k, 3))
i    = 0
k    = 0

In [11]:
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()= 'jue' ]"))).click()
df3 = df3.append(scrap(df, df2, data, i, k, 4))
i    = 0
k    = 0

In [12]:
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()= 'vie' ]"))).click()
df3 = df3.append(scrap(df, df2, data, i, k, 5))
i    = 0
k    = 0

In [13]:
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()= 'sáb' ]"))).click()
df3 = df3.append(scrap(df, df2, data, i, k, 6))
i    = 0
k    = 0

In [14]:
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()= 'dom' ]"))).click()
df3 = df3.append(scrap(df, df2, data, i, k, 7))
i    = 0
k    = 0

In [16]:
df3.to_csv('distribucion_dia_horario.csv', sep=';', index=False)